In [ ]:
#installing packages
%%capture
!pip install torchtext==0.6.0
!pip install datasets


In [ ]:
from datasets import load_dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import json
import random
from tqdm import tqdm
import torch.nn.functional as F
import string
from sklearn.model_selection import train_test_split
import numpy as np


In [ ]:
# Load the dataset
raw_datasets = load_dataset("Sunbird/salt-dataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/25006 [00:00<?, ? examples/s]

In [ ]:
#importing the deep copy package to help copy the hugging face dataset in its dictionary format
from copy import deepcopy

dataset = deepcopy(raw_datasets)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 25006
    })
})

In [ ]:
# Define a function to extract languages
import json

def extract_languages(examples):
    inputs = []
    targets = []
    for example in examples["text"]:
        # Parse the JSON string in each example's "text" field
        example_dict = json.loads(example)
        # Extract English and Luganda texts from the parsed dictionary
        inputs.append(example_dict["English"])
        targets.append(example_dict["Luganda"])
    return {"inputs": inputs, "targets": targets}



In [ ]:
dataset['train'][0]

{'text': '{"English":"Eggplants always grow best under warm conditions.","Luganda":"Bbiringanya lubeerera  asinga kukulira mu mbeera ya bugumu","Runyankole":"Entonga buriijo zikurira omu mbeera y\'obwire erikutagata","Ateso":"Epoloi ebirinyanyi ojok apakio nu emwanar akwap.","Lugbara":"Birinyanya eyi zo kililiru ndeni angu driza ma alia.","Acholi":"Bilinyanya pol kare dongo maber ka lyeto tye"}'}

In [ ]:
# Apply extraction
dataset = dataset.map(extract_languages, batched=True, remove_columns=["text"])


Map:   0%|          | 0/25006 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 25006
    })
})

In [ ]:
type(dataset['train']['inputs'])

list

In [ ]:
source = dataset['train']['inputs']
target = dataset['train']['targets']


In [ ]:
print(source[0])
print(target[0])

Eggplants always grow best under warm conditions.
Bbiringanya lubeerera  asinga kukulira mu mbeera ya bugumu


In [ ]:
#splitting the dataset
split_dataset = dataset['train'].train_test_split(test_size=0.2, seed=20)

In [ ]:
split_dataset

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 20004
    })
    test: Dataset({
        features: ['inputs', 'targets'],
        num_rows: 5002
    })
})

In [ ]:
split_dataset['train']['inputs'][0]

'District officials should pass economic policies that are beneficial to their communities'

In [ ]:
split_dataset['train']['targets'][0]

"Abakulembeze ba disitulikiti balina okuyisa amateeka g'ebyensimbi agaganyula ebitundu byabwe."

In [ ]:
%%capture
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!pip install -U pip transformers
!pip install sentencepiece


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_checkpoint = "facebook/nllb-200-distilled-600M"
# model_checkpoint = "facebook/nllb-200-distilled-1.3B"


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, src_lang="eng_Latn", tgt_lang='lug_Latn', return_tensors="pt")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [ ]:
translator = pipeline("translation", model = model_checkpoint,tokenizer=tokenizer, src_lang="eng_Latn", tgt_lang='lug_Latn', max_length = 400)
translator("come here")

[{'translation_text': 'jjangu wano'}]

In [ ]:
#testing the tokenizer
source_test = split_dataset['train']['inputs'][1]
target_test = split_dataset['train']['targets'][1]


In [ ]:
#testing the tokenizer
inputs = tokenizer(source_test, text_target=target_test)
inputs

{'input_ids': [256047, 1617, 131172, 409, 1151, 4312, 248083, 8910, 349, 190681, 248075, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [256110, 1050, 112272, 123930, 1694, 35, 4033, 583, 67674, 93, 115, 248075, 2]}

In [ ]:
max_length = 400

def preprocess_function(inputs, targets):
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs


In [ ]:
split_dataset['train']

Dataset({
    features: ['inputs', 'targets'],
    num_rows: 20004
})

In [ ]:
tokenized_datasets = split_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=split_dataset["train"].column_names,
    input_columns=["inputs", "targets"],
)


Map:   0%|          | 0/20004 [00:00<?, ? examples/s]

Map:   0%|          | 0/5002 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 20004
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 5002
    })
})

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])

batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels'])

In [ ]:
batch["labels"]

tensor([[256110,   1050, 112272, 123930,   1694,     35,   4033,    583,  67674,
             93,    115, 248075,      2,   -100,   -100,   -100,   -100,   -100],
        [256110, 157509,   1858,   3126, 248116,   4567,   5502, 174709,   3126,
           6104,   3126,  86365,    834,    107,    476, 194036, 248075,      2]])

In [ ]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[256110, 1050, 112272, 123930, 1694, 35, 4033, 583, 67674, 93, 115, 248075, 2]
[256110, 157509, 1858, 3126, 248116, 4567, 5502, 174709, 3126, 6104, 3126, 86365, 834, 107, 476, 194036, 248075, 2]


In [ ]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=2,
)
eval_dataloader = DataLoader(
    tokenized_datasets["test"], collate_fn=data_collator, batch_size=2
)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
#defining the optmizer
from transformers import AdamW

optimizer = AdamW(model.parameters())

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [ ]:
!pip install sacrebleu

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")


  0%|          | 0/30006 [00:00<?, ?it/s]

  0%|          | 0/2501 [00:00<?, ?it/s]

epoch 0, BLEU score: 8.01


  0%|          | 0/2501 [00:00<?, ?it/s]

epoch 1, BLEU score: 14.39


  0%|          | 0/2501 [00:00<?, ?it/s]

epoch 2, BLEU score: 17.83


##This notebook goes through the entire process of training a model that translations English to Luganda and the final bleu score was **17.83**.